In [ ]:
import torch
from collections import Counter
from intersection_over_union import intersection_over_union

#Mean avearge Precision:
the steps:
1.   get all bounding box predictions on our test set
2.   sort the descending confidence scores
3.   calcultate the precision and recall as we go through all outputs.
4.  plot a graph of the precision in relation to the recall.
5.  calculate the Area nnder the previous graph.
6.  this process should be done for all classes.
7.  all this step are done for the threshold of 0.5, we need to this calculations for many IOUs "0.5,0.55,0.6,...,0.95" , and finaally average them to get mAP@0.5:0.05:0.95.




In [ ]:
#define a function to calculate mean average precision (mAP)
def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5, box_format="midpoint", num_classes=20):
  # This Function takes the following parameters:
  #   1- pred_boxes: list of lists that containis predicted bboxes with the folllowing format for each bbox [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
  #   2- true_boxes: the same as above but for ground-truth bboxes
  #   3- iou_threshold: threshold to determin if a predicted bbox is correct or not
  #   4- box_format: either "midpoint" or "corners"
  #   5- num_classes: number of classes
  # and return the mAP across all classes 

  # list storing all AP for respective classes
  average_precisions = []
  # used for numerical stability later on
  epsilon = 1e-6
  #to calculate for each class
  for c in range(num_classes):
      detections = []
      ground_truths = []
      # Go through all predictions and targets,
      # and only add the ones that belong to the
      # current class c
      for detection in pred_boxes:
          if detection[1] == c:
              detections.append(detection)
      for true_box in true_boxes:
          if true_box[1] == c:
              ground_truths.append(true_box)
      # find the amount of bboxes for each training example
      # Counter here finds how many ground truth bboxes we get
      # for each training example, so let's say img 0 has 3,
      # img 1 has 5 then we will obtain a dictionary with:
      # amount_bboxes = {0:3, 1:5}
      amount_bboxes = Counter([gt[0] for gt in ground_truths])
      # We then go through each key, val in this dictionary
      # and convert to the following (w.r.t same example):
      # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
      for key, val in amount_bboxes.items():
          amount_bboxes[key] = torch.zeros(val)
      # sort by box probabilities which is index 2
      detections.sort(key=lambda x: x[2], reverse=True)
      TP = torch.zeros((len(detections)))
      FP = torch.zeros((len(detections)))
      total_true_bboxes = len(ground_truths)
      # If none exists for this class then we can safely skip
      if total_true_bboxes == 0:
          continue
      for detection_idx, detection in enumerate(detections):
          # Only take out the ground_truths that have the same
          # training idx as detection
          ground_truth_img = [bbox for bbox in ground_truths if bbox[0] == detection[0]]
          num_gts = len(ground_truth_img)
          best_iou = 0
          for idx, gt in enumerate(ground_truth_img):
              iou = intersection_over_union(torch.tensor(detection[3:]),torch.tensor(gt[3:]),box_format=box_format,)
              if iou > best_iou:
                  best_iou = iou
                  best_gt_idx = idx
          #so now we have taken out a single bounding box for a particular class in a particular
          #image, and we have taken all the ground truth bounding boxes for that particular image
          #and we compared that bounding box between all the target bounding boxes and we have checked
          #the IOU's between all of them and kept track of the best iou

          #check if the bect iou is greated that the threshold then the predictions is correct
          if best_iou > iou_threshold:
              # only detect ground truth detection once
              if amount_bboxes[detection[0]][best_gt_idx] == 0:
                  # true positive and add this bounding box to seen
                  TP[detection_idx] = 1
                  amount_bboxes[detection[0]][best_gt_idx] = 1
              else:
                  FP[detection_idx] = 1

          # if IOU is lower then the detection is a false positive
          else:
              FP[detection_idx] = 1
      TP_cumsum = torch.cumsum(TP, dim=0)
      FP_cumsum = torch.cumsum(FP, dim=0)
      recalls = TP_cumsum / (total_true_bboxes + epsilon)
      precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
      precisions = torch.cat((torch.tensor([1]), precisions))
      recalls = torch.cat((torch.tensor([0]), recalls))
      # torch.trapz for numerical integration
      average_precisions.append(torch.trapz(precisions, recalls))
  return sum(average_precisions) / len(average_precisions)
